In [1]:
!sudo apt install subversion
!svn checkout https://github.com/microsoft/recommenders/trunk/reco_utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1
Suggested packages:
  db5.3-util libapache2-mod-svn subversion-tools
The following NEW packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1 subversion
0 upgraded, 5 newly installed, 0 to remove and 33 not upgraded.
Need to get 2,237 kB of archives.
After this operation, 9,910 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libapr1 amd64 1.6.3-2 [90.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libaprutil1 amd64 1.6.1-2 [84.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libserf-1-1 amd64 1.3.9-6 [44.4 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe

In [1]:
import logging
import numpy as np
import pandas as pd

from reco_utils.dataset.python_splitters import python_stratified_split
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from reco_utils.recommender.sar import SAR

In [3]:
!wget http://www.occamslab.com/petricek/data/ratings.dat

--2020-06-30 14:35:27--  http://www.occamslab.com/petricek/data/ratings.dat
Resolving www.occamslab.com (www.occamslab.com)... 82.208.6.71
Connecting to www.occamslab.com (www.occamslab.com)|82.208.6.71|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274354671 (262M) [text/plain]
Saving to: ‘ratings.dat’

ratings.dat         100%[===================>] 261.64M   563KB/s    in 8m 43s  

2020-06-30 14:44:11 (512 KB/s) - ‘ratings.dat’ saved [274354671/274354671]



In [2]:
# top k items to recommend
TOP_K = 10
data = pd.read_csv('ratings.dat', names=['user_id', 'profile_id', 'rating'])[:50000]
data['rating'] = data['rating'].astype(np.float32)

In [3]:
train, test = python_stratified_split(data, ratio=0.75, col_user='user_id', col_item='profile_id', seed=42)

In [4]:
model = SAR(
    col_user="user_id",
    col_item="profile_id",
    col_rating="rating",
    similarity_type="jaccard",
    time_decay_coefficient=0,
    normalize=True
)

In [5]:
model.fit(train)

In [6]:
top_k = model.recommend_k_items(test, remove_seen=True, normalize=True) # if we want just top K for users
score = model.score(test, remove_seen=True, normalize=True) # if we want to predict all items for users
predictions = model.predict(test) # if we want to predict the test set only, MAYBE NOT NORMALIZED TO SCALE ORIGINAL RATINGS!

/content/reco_utils/recommender/sar/sar_singlenode.py:333: RuntimeWarning: invalid value encountered in true_divide
  self.unity_user_affinity[user_ids, :].dot(self.item_similarity),
/content/reco_utils/recommender/sar/sar_singlenode.py:333: RuntimeWarning: invalid value encountered in true_divide
  self.unity_user_affinity[user_ids, :].dot(self.item_similarity),
Items found in test not seen during training, new items will have score of 0


In [7]:
top_k.head()

,user_id,profile_id,prediction
0,1,71704,10.000001
1,1,93927,10.000000
2,1,97383,10.000000
3,1,168635,10.000000
4,1,129043,10.000000


In [10]:
predictions

,user_id,profile_id,prediction
0,1,125537,0.000000
1,1,144414,2.177778
2,1,84587,20.721573
3,1,169128,0.000000
4,1,71594,23.242857
...,...,...,...
12493,325,7242,19.314894
12494,325,10178,0.000000
12495,325,26596,0.000000
12496,325,40551,0.000000
